WeatherPy

In [31]:
import requests
import json
from pprint import pprint
from api_keys import weather_api_key
from citipy import citipy

In [32]:
url = "https://api.openweathermap.org/data/2.5/weather?"
lat = 34.0901
lon = -118.4065
units = "Imperial"

q_url = f'{url}lat={lat}&lon={lon}&units={units}&APPID={weather_api_key}'
# https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API key}

In [33]:
response = requests.get(q_url).json()
pprint(response)

{'base': 'stations',
 'clouds': {'all': 0},
 'cod': 200,
 'coord': {'lat': 34.0901, 'lon': -118.4065},
 'dt': 1650241260,
 'id': 5328041,
 'main': {'feels_like': 68.32,
          'humidity': 59,
          'pressure': 1013,
          'temp': 68.95,
          'temp_max': 79.72,
          'temp_min': 58.06},
 'name': 'Beverly Hills',
 'sys': {'country': 'US',
         'id': 2043049,
         'sunrise': 1650201587,
         'sunset': 1650248765,
         'type': 2},
 'timezone': -25200,
 'visibility': 10000,
 'weather': [{'description': 'clear sky',
              'icon': '01d',
              'id': 800,
              'main': 'Clear'}],
 'wind': {'deg': 260, 'speed': 13.8}}


In [43]:
temp = response["main"]["temp"]
humidity = response["main"]["humidity"]
cloudiness = response["clouds"]["all"]
wind_speed = response["wind"]["speed"]
city = citipy.nearest_city(lat, lon)
f_city = city.city_name.title()
f_country = city.country_code.upper()
print(f'{f_city}, {f_country}')

Beverly Hills, US
